In [25]:
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import numba
from sklearn.preprocessing import MinMaxScaler



In [20]:

df = 'D:/Python/Hydraulic Rig Dataset/Data/sensor_data_df.pkl'
target_df ='D:/Python/Hydraulic Rig Dataset/Data/profile_df.pkl'


target_data = pd.read_pickle(target_df)
data = pd.read_pickle(df)
data.dropna(inplace=True)
target_data.dropna(inplace=True)
print(data.head(5))
print(target_data.columns)


     CE_1    CE_2    CE_3    CE_4    CE_5    CE_6    CE_7    CE_8    CE_9  \
0  47.202  47.273  47.250  47.332  47.213  47.372  47.273  47.438  46.691   
1  29.208  28.822  28.805  28.922  28.591  28.643  28.216  27.812  27.514   
2  23.554  23.521  23.527  23.008  23.042  23.052  22.658  22.952  22.908   
3  21.540  21.419  21.565  20.857  21.052  21.039  20.926  20.912  20.989   
4  20.460  20.298  20.350  19.867  19.997  19.972  19.924  19.813  19.691   

    CE_10  ...  VS1_51  VS1_52  VS1_53  VS1_54  VS1_55  VS1_56  VS1_57  \
0  46.599  ...   0.554   0.552   0.545   0.553   0.553   0.539   0.544   
1  27.481  ...   0.555   0.547   0.548   0.544   0.536   0.542   0.540   
2  22.359  ...   0.543   0.544   0.543   0.554   0.544   0.544   0.545   
3  20.882  ...   0.549   0.538   0.553   0.543   0.553   0.555   0.544   
4  19.634  ...   0.546   0.546   0.544   0.552   0.539   0.540   0.549   

   VS1_58  VS1_59  VS1_60  
0   0.545   0.535   0.543  
1   0.533   0.531   0.534  
2   0.54

In [21]:
df_combined =pd.concat([data, target_data], axis=1)
print(df_combined.head(5))

     CE_1    CE_2    CE_3    CE_4    CE_5    CE_6    CE_7    CE_8    CE_9  \
0  47.202  47.273  47.250  47.332  47.213  47.372  47.273  47.438  46.691   
1  29.208  28.822  28.805  28.922  28.591  28.643  28.216  27.812  27.514   
2  23.554  23.521  23.527  23.008  23.042  23.052  22.658  22.952  22.908   
3  21.540  21.419  21.565  20.857  21.052  21.039  20.926  20.912  20.989   
4  20.460  20.298  20.350  19.867  19.997  19.972  19.924  19.813  19.691   

    CE_10  ...  VS1_52  VS1_53  VS1_54  VS1_55  VS1_56  VS1_57  VS1_58  \
0  46.599  ...   0.552   0.545   0.553   0.553   0.539   0.544   0.545   
1  27.481  ...   0.547   0.548   0.544   0.536   0.542   0.540   0.533   
2  22.359  ...   0.544   0.543   0.554   0.544   0.544   0.545   0.544   
3  20.882  ...   0.538   0.553   0.543   0.553   0.555   0.544   0.543   
4  19.634  ...   0.546   0.544   0.552   0.539   0.540   0.549   0.542   

   VS1_59  VS1_60  Target  
0   0.535   0.543       1  
1   0.531   0.534       1  
2   0.53

In [22]:
X = df_combined.drop(columns=['Target'])
y = df_combined['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

print(X.dtypes)
print(y.dtypes)

X_train_np = X_train.to_numpy()  # Convert to NumPy array
X_test_np = X_test.to_numpy()


CE_1      float64
CE_2      float64
CE_3      float64
CE_4      float64
CE_5      float64
           ...   
VS1_56    float64
VS1_57    float64
VS1_58    float64
VS1_59    float64
VS1_60    float64
Length: 43680, dtype: object
int64


In [23]:
# Apply MiniROCKET (multivariate version)
rocket = MiniRocketMultivariate(num_kernels=10000)
rocket.fit(X_train_np)
X_train_transformed = rocket.transform(X_train_np)
X_test_transformed = rocket.transform(X_test_np)

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_transformed)
X_test_scaled = scaler.transform(X_test_transformed)